# Game Genres Notebook

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2 as pg
import pandas.io.sql as pd_sql
from copy import deepcopy
import numpy as np

from fuzzywuzzy import process

In [47]:
# Postgres info to connect

connection_args = {
 "host": "twitchdata.chd4n5ul8muk.us-east-2.rds.amazonaws.com",
 "user": "postgres",
   "password":"FwwBFmleh65qYxKxDVb9",
 "port": 5432,
 "dbname": "twitchdata"
}

# We will talk about this magic Python trick!
connection = pg.connect(**connection_args)

In [48]:
query = '''
SELECT * FROM game_genres ORDER BY game_name;
'''
df=pd_sql.read_sql(query, connection)

In [14]:
print(df.columns)
df.shape

Index(['game_name', 'game_genres'], dtype='object')


(2173, 2)

In [49]:
df.drop_duplicates(inplace = True)

In [52]:
genres = df['game_genres'].unique()

In [16]:
df.loc[df['game_genres'] == 'IRL']

,game_name,game_genres
84,ASMR,IRL
85,ASMR,IRL
173,Beauty & Body Art,IRL
687,Food & Drink,IRL
690,Food & Drink,IRL
933,Just Chatting,IRL
934,Just Chatting,IRL
1050,Makers & Crafting,IRL
1195,Music & Performing Arts,IRL
1197,Music & Performing Arts,IRL


# Goal:
Given a string first:
- Fuzzy match any titles
    - if title matches, show game and related games
- if string does not fuzzy match any titles
    - find similarity of string to all genres
    - take the top 3 similarities and show them?
    

In [50]:
# query = '''SELECT DISTINCT(game_name) FROM game_genres ORDER BY game_name;'''
# df=pd_sql.read_sql(query, connection)

In [20]:
# df.drop_duplicates(inplace = True)

In [22]:
df.head()

,game_name
0,140
1,2Dark
2,7 Days to Die
3,8-Bit Armies
4,Agar.io


In [27]:
list(df['game_name'])

['140',
 '2Dark',
 '7 Days to Die',
 '8-Bit Armies',
 'Agar.io',
 'Age of Empires II',
 'Age of Empires III: The Asian Dynasties',
 'Age of Mythology',
 'A Hat in Time',
 'Aion',
 'Alan Wake',
 'Albion Online',
 'Alice: Madness Returns',
 'Alien: Isolation',
 'Always On',
 'American Fugitive',
 'American Truck Simulator',
 "America's Army: Proving Grounds",
 'Amnesia: The Dark Descent',
 'Anarchy Online',
 'Animal Crossing',
 'Animal Crossing: New Leaf',
 'Anime Land',
 'Anno 1800',
 'Anthem',
 'Antichamber',
 'AO Tennis',
 'APB Reloaded',
 'Apex Legends',
 'A Plague Tale: Innocence',
 'Ara Fell',
 'ArcheAge',
 'Arena of Evolution: Red Tides',
 'Arena of Valor',
 'ARK',
 'Arknights',
 'Arma 3',
 'ArmA II: Operation Arrowhead',
 'Art',
 'Artifact',
 'Ashen',
 'ASMR',
 "Assassin's Creed: Brotherhood",
 "Assassin's Creed Chronicles: China",
 "Assassin's Creed II",
 "Assassin's Creed III",
 "Assassin's Creed IV: Black Flag",
 "Assassin's Creed Odyssey",
 "Assassin's Creed Origins",
 "Assas

In [63]:
len(genres)

38

In [89]:
list(genres)

['Rhythm & Music Game',
 'Platformer',
 'Adventure Game',
 'Simulation',
 'Shooter',
 'FPS',
 'Strategy',
 'RTS',
 'NONE',
 'Indie Game',
 'MMORPG',
 'Horror',
 'Action',
 'Open World',
 'Driving/Racing Game',
 'Puzzle',
 'RPG',
 'Stealth',
 'Sports Game',
 'MOBA',
 'Creative',
 'Card & Board Game',
 'Survival',
 'IRL',
 "Shoot 'Em Up",
 'Roguelike',
 'Fighting',
 'Compilation',
 'Hidden Objects',
 'Series: Souls',
 'Visual Novel',
 'Flight Simulator',
 'Metroidvania',
 'Educational Game',
 'Point and Click',
 'RETROGAMEPLACEHOLDER',
 'Pinball',
 'Gambling Game']

In [94]:
def match_string(str2Match):
    
    strOptions = list(df['game_name'])
    game_ratios = process.extract(str2Match,strOptions,limit=10)
    print("Game ratios: ", game_ratios)
    # You can also select the string with the highest matching percentage
    # highest = process.extractOne(str2Match,strOptions)
    strOptions = list(genres)

    genre_ratios = process.extract(str2Match,strOptions,limit=10)
    print("Genre ratios: ", genre_ratios)
    return game_ratios,genre_ratios

In [96]:
match_string('None')

Game ratios:  [('Albion Online', 68), ('A Plague Tale: Innocence', 68), ('Assetto Corsa Competizione', 68), ('Assetto Corsa Competizione', 68), ('Astroneer', 68), ('Astroneer', 68), ('Bloodborne', 68), ('Bloodborne', 68), ('Bloodborne', 68), ('Clone Drone in the Danger Zone', 68)]
Str2match:  None
Genre ratios:  [('NONE', 100), ('Visual Novel', 68), ('Simulation', 51), ('Action', 51), ('Platformer', 45), ('Shooter', 45), ('Indie Game', 45), ('Open World', 45), ('Compilation', 45), ('Hidden Objects', 45)]


([('Albion Online', 68),
  ('A Plague Tale: Innocence', 68),
  ('Assetto Corsa Competizione', 68),
  ('Assetto Corsa Competizione', 68),
  ('Astroneer', 68),
  ('Astroneer', 68),
  ('Bloodborne', 68),
  ('Bloodborne', 68),
  ('Bloodborne', 68),
  ('Clone Drone in the Danger Zone', 68)],
 [('NONE', 100),
  ('Visual Novel', 68),
  ('Simulation', 51),
  ('Action', 51),
  ('Platformer', 45),
  ('Shooter', 45),
  ('Indie Game', 45),
  ('Open World', 45),
  ('Compilation', 45),
  ('Hidden Objects', 45)])